<a href="https://colab.research.google.com/github/maxforsjoki/quinyx_assignment/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I first solve Part 2 with the code in the notebook below. At the end I include a short evaluation of the work in Part 3

In [133]:
#Imports
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error

In [1]:

#Importing data
from google.colab import files

uploaded = files.upload()

Saving VesselData.xlsx to VesselData.xlsx


In [104]:
df = pd.read_excel('VesselData.xlsx')
df.head(10)

In [105]:
#Inspecting some data
print(len(df["vesseltype"].unique()))
print(len(df["vesseldwt"].unique()))
print(len(df["stevedorenames"].unique()))
print(len(df["previousportid"].unique()))
print(len(df["nextportid"].unique()))
print(len(df["vesselid"].unique()))

5
2505
1375
692
569
3022


In [ ]:
#Fixing categorical variables, traveltype and vesseltype as well as new columns for the total transships

one_hot = pd.get_dummies(df['vesseltype'])
df = df.drop('vesseltype',axis = 1)
df = df.join(one_hot)
one_hot = pd.get_dummies(df['traveltype'])
df = df.drop('traveltype',axis = 1)
df = df.join(one_hot)

for i in range(1,5):
  df["transship{}".format(i)] = df["discharge{}".format(i)]+df["load{}".format(i)]

df.columns

In [113]:
#Deciding which variables we include in the predictors, that is our X and y in the regression model
predictors = ["vesseldwt", 1.0, 2.0, 3.0, 4.0, 5.0, "ARRIVAL", "SHIFT"]
predicted = ["transship1","transship2","transship3","transship4"]

In [114]:
#Data for regression, y = transships and X are the predictors defined earlier
y = df[predicted].to_numpy()
X = df[predictors].to_numpy()
print(X.shape)
print(y.shape)

(8208, 8)
(8208, 4)


In [115]:
#Removing rows in X or y with NaN
remove_rows1 = np.isnan(X).any(axis=1)
remove_rows2 = np.isnan(y).any(axis=1)
remove_rows = np.logical_or(remove_rows1,remove_rows2)
print(len([i for i, x in enumerate(remove_rows) if x]))
X = np.delete(X,remove_rows,0)
y = np.delete(y,remove_rows,0)

2


In [116]:
#Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [118]:
#Normalizing based on training data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [127]:
#Creating a simple regression neural network
n_features = len(predictors)
n_outputs = 4

model = Sequential()
model.add(Dense(50, input_dim = n_features, activation='relu'))
model.add(Dense(20, activation = 'relu'))
model.add(Dense(n_outputs, activation = 'linear'))

model.compile(loss='mse', optimizer='adam')


In [ ]:
#Training model
history = model.fit(X_train, y_train, epochs=400, batch_size=16, verbose=2, validation_split=0.2)

In [136]:
#Evaluating results on test data

yhat = model.predict(X_test)
error = mean_squared_error(y_test, yhat)
print("Mean Squared Error: {}".format(error))


Mean Squared Error: 139020561.3096075


Part 3:

I chose to view the problem as a regression problem, where the task was to predict four values given predictors chosen by me. Due to the time limitation, I did not have time to properly inspect the different variables and settled for a simple model using what seemed the most reasonable to me at the time. That is, I chose to include the vesseltype, traveltype and vessel deadweight tonnage as variables and created a 4-dimensional output of the regression model corresponding to the total transship.

Clearly the model did a very poor job. The reason for this is most likely that the y-variables are often large for one cargo type while zero for all the others, as well as often zero for all. Probably the transship data should be seen as categorical in an initial stage, for instance it could've been defined as "coal", "oil", "ore", "petroleum", "mixed cargo types" or "no cargo type". After that a second stage regression model could've been created that predicted the value based on the parameters and an assumption of which type of cargo we are dealing with. If I had more time, I would've attempted to implement this.

As for issues with overfitting, I did not have time to focus on that. It is likely that my model is too complex and has been trained for too many epochs. But clearly the large error is due to some other issues than overfitting, as the training error is also very large.